In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

import matplotlib.pyplot as plt

# Loading data

In [2]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [3]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
model_bis = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

# Selecting/Generating Samples

2.5725006639080936

In [6]:
from lipschitz_decomon_tools import get_local_maximum, echantillonner_boule_l2_simple, square_backward_bounds, function_to_optimize_all


In [7]:
pt_choosen = 55
x = images[pt_choosen:pt_choosen+1].flatten().detach().cpu().numpy()
label = labels[pt_choosen:pt_choosen+1]
eps=0.5
nb_pts = 10

We generate nb_pts random points within the ball.

In [8]:
y_list = []
for i in range(nb_pts):
    ech = echantillonner_boule_l2_simple(x, eps)
    print(np.linalg.norm(x-ech))
    y_list.append(ech)

0.06370150924629762
0.4514828456636468
0.03849168362062583
0.21818509481831644
0.09248876890841125
0.46502662886661633
0.1854948136066631
0.18653074939382672
0.25169979962137257
0.4168406969150333


# Solving

We solve our problem with these 10 random points

In [9]:
x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)

In [10]:
f_adv

10.673187277227058

In [ ]:
# from radius_evaluation_tools import single_compute_relaxation_radius

In [13]:
# eps_working = single_compute_relaxation_radius(pt_choosen, images, labels, model, nb_pts)

In [14]:
# eps_working

In [31]:
# y_list = [x]
# f_list = [get_local_maximum(x, label, eps, y_list, model)[1]]
# for i in range(10):
#     print(i)
#     i = 2*i
#     nb_pts = i+1
#     y_list.append(echantillonner_boule_l2_simple(x, eps))
#     x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)
#     f_list.append(f_adv)

# list_i = []
# for i in range(3):
#     list_i.append(2*i)


# plt.scatter(list_i, f_list)
# plt.xlabel("nb yi")
# plt.ylabel("max function")
# plt.show()

In [13]:
x_ech = echantillonner_boule_l2_simple(x, eps)
print(np.linalg.norm(x-x_ech))
x_ball_center = np.asarray(x, dtype=np.float64)

l = x-eps
u = x+eps

W_list = []
b_list = []
for y_i in y_list:
    # W, b = square_backward_bounds(l,u,y_i)
    W, b = square_backward_bounds(l,u,y_i)
    print(b)
    W_list.append(W)
    b_list.append(b)
    
function_to_optimize_all(x_ech, label, W_list, b_list, y_list, model)

0.3454614151835286
[195.98196976]
[196.19583053]
[195.98540864]
[195.88200628]
[195.94679058]
[195.86485926]
[195.79642287]
[196.13448578]
[196.02867958]
[196.00767629]


10.681619765090458

## Pyiavski

In [92]:
y_list = []
y_list.append(x)

for i in range(20):
    x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)
    y_list.append(x_adv)

In [94]:
f_adv

23.89740562438965

In [44]:
np.sum(W_list[5] - W_list[1] != 0)

784

# Tests

In [14]:
import pdb

In [15]:
def square_backward_bounds_reduced(l, u, y):
    # l (4,)
    # u (4,)
    # y (4,)
    l = l[151:153]
    u = u[151:153]
    y = y[151:153]
    print(l, u, y)
    u = u - y
    l = l - y
    print(u, l)

    W = u + l #(4,)
    print(W)
    b = np.sum(-u*l) - W@y #scalar
    # pdb.set_trace()
    return W, np.array(b)[None]#(4,) & (1,)

In [16]:
v = x-eps
idx = np.nonzero(x)

In [17]:
idx

(array([145, 146, 147, 148, 173, 174, 175, 176, 177, 182, 183, 184, 185,
        186, 202, 203, 204, 205, 206, 210, 211, 212, 213, 214, 215, 216,
        217, 230, 231, 232, 233, 234, 238, 239, 240, 241, 242, 243, 244,
        245, 246, 260, 261, 262, 263, 264, 268, 269, 270, 271, 272, 273,
        274, 289, 290, 291, 292, 293, 299, 300, 301, 302, 318, 319, 320,
        321, 322, 326, 327, 328, 329, 330, 346, 347, 348, 349, 350, 353,
        354, 355, 356, 357, 375, 376, 377, 378, 379, 380, 381, 382, 383,
        384, 385, 404, 405, 406, 407, 408, 409, 410, 411, 412, 432, 433,
        434, 435, 436, 437, 438, 458, 459, 460, 461, 462, 463, 464, 465,
        466, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 513, 514,
        515, 516, 517, 519, 520, 521, 522, 541, 542, 543, 544, 545, 548,
        549, 550, 569, 570, 571, 572, 576, 577, 578, 579, 597, 598, 599,
        600, 604, 605, 606, 607, 625, 626, 627, 628, 629, 630, 631, 632,
        633, 634, 635, 654, 655, 656, 657, 658, 659

In [18]:
y_list[5][145: 147]

array([0.63515117, 0.98249315])

In [19]:
x[145: 147]

array([0.63529414, 0.99215686], dtype=float32)

In [20]:
W1, b1 = square_backward_bounds_reduced(x-eps, x+eps, y_list[2])

[-0.5 -0.5] [0.5 0.5] [0.00628395 0.00533263]
[0.49371605 0.49466737] [-0.50628395 -0.50533263]
[-0.0125679  -0.01066527]


In [22]:
W3, b3 = square_backward_bounds_reduced(np.asarray(x, dtype=np.float32)-eps, np.asarray(x, dtype=np.float32)+eps, y_list[2])

[-0.5 -0.5] [0.5 0.5] [0.00628395 0.00533263]
[0.49371605 0.49466737] [-0.50628395 -0.50533263]
[-0.0125679  -0.01066527]


In [23]:
W2, b2 = square_backward_bounds_reduced(np.asarray(x, dtype=np.float64)-eps, np.asarray(x, dtype=np.float64)+eps, y_list[5])

[-0.5 -0.5] [0.5 0.5] [-0.00148758 -0.01061017]
[0.50148758 0.51061017] [-0.49851242 -0.48938983]
[0.00297516 0.02122035]


In [62]:
W1

array([ 0.01725835, -0.00543194])

In [63]:
W1

array([ 0.01725835, -0.00543194])

In [64]:
x_ech = echantillonner_boule_l2_simple(x, eps)

In [65]:
function_to_optimize_all(x_ech, label, [W1, W2], [b1, b2], y_list[151:153], model)

UnboundLocalError: local variable 'function' referenced before assignment

In [24]:
outputs = []
for i in range(len(y_list)):
    output = model(y_list[i].reshape((1,28,28))[None]).cpu().detach().numpy()[0,0] +\
                1*np.sqrt(W_list[i]@x+b_list[i]) #scalar
    outputs.append(output)
function = np.min(outputs)

In [25]:
function

10.681383384238005

## Test comparison function without relaxation vs lip certificate

In [72]:
nb_pts = 1000

In [73]:
pt_choosen = 1
# eps = np.abs(model(images[pt_choosen:pt_choosen+1]).detach().cpu().numpy())
eps = 3.4
x = images[pt_choosen:pt_choosen+1].flatten().detach().cpu().numpy()
label = labels[pt_choosen:pt_choosen+1]

y_list = []
for i in range(nb_pts):
    ech = echantillonner_boule_l2_simple(x, eps)
    # print(np.linalg.norm(x-ech))
    y_list.append(ech)

In [74]:
def f_bar(z):
    outputs = []
    for i in range(len(y_list)):
        output = model(y_list[i].reshape((1,28,28))[None]).cpu().detach().numpy()[0,0] +\
                    1*np.sqrt(np.sum(np.square(z - y_list[i]))) #scalar
        outputs.append(output)
    function = np.min(outputs)
    return function

In [ ]:
np.max([f_bar(echantillonner_boule_l2_simple(x, eps, uniform=False)) for _ in range(100)])

pour nb point donné quel est le rayon empirique obtenu : faire meme tableau que slides